In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, PowerNorm

import numpy as np

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.table import Table, QTable, hstack
from astropy.io import fits

import logging

In [2]:
from gammapy.data import Observation
from gammapy.datasets import SpectrumDataset, Datasets, FluxPointsDataset
from gammapy.estimators import LightCurveEstimator
from gammapy.irf import (
    EffectiveAreaTable2D,
    Background2D, Background3D,
    EnergyDispersion2D, EDispKernel, EDispKernelMap,
)
from gammapy.makers import SpectrumDatasetMaker
from gammapy.maps import MapAxis, RegionGeom, TimeMapAxis
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    ExpCutoffPowerLawSpectralModel,
    SkyModel,
    LightCurveTemplateTemporalModel
)
from gammapy.utils.random import get_random_state

In [ ]:
COSI_lightcurves = QTable.read(, format = 'fits')

In [ ]:
def plot_light_curve(avg, figsize = (15.0, 22.0),curve_index = 0):
    
    # Time
    SIM_time_val = starting_times + livetimes / 2.0 - trigger_time_t0
    SIM_time_val = SIM_time_val.sec
    SIM_time_wid = livetimes.value
    
    if avg:
        # Count rates
        SIM_cnt_rate = AVG_cnt_rate
        SIM_cnt_erro = AVG_cnt_erro
        
        # Background rates    
        SIM_bkg_rate = AVG_bkg_rate
        SIM_bkg_erro = AVG_bkg_erro
        
        # Excess Rates
        SIM_exc_rate = AVG_exc_rate
        SIM_exc_erro = AVG_exc_erro
        
        # Predicted Signal Rates
        SIM_exc_pred = AVG_exc_pred
    
    else:
        # Choose one Light Curve
        datasets = List_of_Datasets[curve_index].info_table()
        
        # Count rates
        SIM_cnt_rate = datasets['counts_rate'].value
        SIM_cnt_erro = SIM_cnt_rate / np.sqrt(datasets['counts'].value)
        
        # Background rates    
        SIM_bkg_rate = datasets['background_rate'].value
        SIM_bkg_erro = SIM_bkg_rate / np.sqrt(datasets['background'].value)
        
        # Excess Rates
        SIM_exc_rate = datasets['excess_rate'].value
        SIM_exc_erro = SIM_cnt_erro + SIM_bkg_erro
        
        # Predicted Signal Rates
        SIM_exc_pred = datasets['npred_signal'].value / SIM_time_wid     
    
    ###################################################################
    # Define pyplot Figure and Axes
    fig, axs = plt.subplots(3,
                            figsize = figsize,
                            gridspec_kw = {'height_ratios': [2.5, 2.5, 1]}
                           )
    
    # DATA PLOT
    
    # Plot GBM Data + Error
    axs[0].step(GBM_time_val,
                GBM_cnt_rate,
                label = 'GBM Count rates', color = 'C1', where = 'mid'
               )
    axs[0].bar(x      = GBM_time_val,
               height = GBM_cnt_erro * 2.0,
               bottom = GBM_cnt_rate - GBM_cnt_erro,
               width  = GBM_time_wid,
               align  = 'center', color = 'C1', alpha = 0.5
              )
        
    # Plot GBM Background + Error
    axs[0].step(GBM_time_val,
                GBM_bkg_rate,
                label = 'GBM bkgd rates', color = 'C3', where = 'mid'
               )
    axs[0].bar(x      = GBM_time_val,
               height = GBM_bkg_erro * 2.0,
               bottom = GBM_bkg_rate - GBM_bkg_erro,
               width  = GBM_time_wid,
               align  = 'center', color = 'C3', alpha = 0.5
              )
        
    # Plot Simulated Light Curve + Error
    axs[0].step(SIM_time_val,
                SIM_cnt_rate,
                label = 'Simulated Count rates', color = 'C0', where = 'mid'
               )
    axs[0].bar(x      = SIM_time_val,
               height = SIM_cnt_erro * 2.0,
               bottom = SIM_cnt_rate - SIM_cnt_erro,
               width  = SIM_time_wid,
               align  = 'center', color = 'C0', alpha = 0.5
              )
        
    # Plot Simulated Background + Error
    axs[0].step(SIM_time_val,
                SIM_bkg_rate,
                label = 'Simulated Bkgd rates', color = 'b', where = 'mid'
               )
    axs[0].bar(x      = SIM_time_val,
               height = SIM_bkg_erro * 2.0,
               bottom = SIM_bkg_rate - SIM_bkg_erro,
               width  = SIM_time_wid,
               align  = 'center', color = 'b', alpha = 0.5
              )
    
    # EXCESS PLOT
    
    # Plot GBM Excess rates   
    axs[1].step(GBM_time_val,
                GBM_exc_rate,
                label = 'GBM Excess rates', color = 'C1', where = 'mid'
               )
    axs[1].bar(x = GBM_time_val,
               height = GBM_exc_erro * 2.0,
               bottom = GBM_exc_rate - GBM_exc_erro,
               width  = GBM_time_wid,
               align  = 'center', color = 'C1', alpha = 0.5
              )
        
    # Plot GBM excess prediction: Best fit model
    axs[1].plot(GBM_time_val,
                GBM_exc_pred,
                label = 'GBM Best Fit Model', color = 'C3'
               )
        
    # Plot Simulated Exccess rates
    axs[1].step(SIM_time_val,
                SIM_exc_rate,
                label = 'Simulated Excess rates', color = 'C0', where = 'mid'
               )
    axs[1].bar(x = SIM_time_val,
               height = SIM_exc_erro * 2.0,
               bottom = SIM_exc_rate - SIM_exc_erro,
               width  = SIM_time_wid,
               align  = 'center', color = 'C0', alpha = 0.5
              )
                
    # Plot Predicted Rate (Gammapy model through IRFs)
    axs[1].plot(SIM_time_val,
                SIM_exc_pred,
                label = 'Gammapy Model Predicted Excess', color = 'b'
               )
        
    # BACKGROUND ZOOM    
    
    # Plot GBM Background + Error
    axs[2].step(GBM_time_val,
                GBM_bkg_rate,
                label = 'GBM bkgd rates', color = 'C3', where = 'mid'
               )
    axs[2].bar(x      = GBM_time_val,
               height = GBM_bkg_erro * 2.0,
               bottom = GBM_bkg_rate - GBM_bkg_erro,
               width  = GBM_time_wid,
               align  = 'center', color = 'C3', alpha = 0.5
              )
        
    # Plot Simulated Background + Error
    axs[2].step(SIM_time_val,
                SIM_bkg_rate,
                label = 'Simulated Bkgd rates', color = 'b', where = 'mid'
               )
    axs[2].bar(x      = SIM_time_val,
               height = SIM_bkg_erro * 2.0,
               bottom = SIM_bkg_rate - SIM_bkg_erro,
               width  = SIM_time_wid,
               align  = 'center', color = 'b', alpha = 0.5
              )
    
    # Plot Mean of the simulated Background    
    axs[2].hlines(y = AVG_bkg_pred,
                  xmin = SIM_time_val[0],
                  xmax = SIM_time_val[-1],
                  linewidth = 2, color = 'C2', label = 'Gammapy predicted background'
                 )    
    
    # LABELS
    axs[0].set_xlabel('Time since trigger (s)', fontsize = 'large')
    axs[1].set_xlabel('Time since trigger (s)', fontsize = 'large')
    axs[2].set_xlabel('Time since trigger (s)', fontsize = 'large')
    axs[0].set_ylabel('Count rates (cts/s)', fontsize = 'large')
    axs[1].set_ylabel('Count rates (cts/s)', fontsize = 'large')
    axs[2].set_ylabel('Count rates (cts/s)', fontsize = 'large')
    
    axs[0].set_xlim(SIM_time_val[0], SIM_time_val[-1])
    axs[1].set_xlim(SIM_time_val[0], SIM_time_val[-1])
    axs[2].set_xlim(SIM_time_val[0], SIM_time_val[-1])
    
    # TITLE PLOT
    if avg:
        plot_title = f'Average of {Number_of_LightCurves} simulated light curves. '
        
    else:
        plot_title = f'Lightcurve simulation {curve_index+1}/{Number_of_LightCurves}. '
        
    plot_title += "Data of "+ title_name_transient
    plot_title += " from "  + title_name_instrument +'.'
    
    axs[0].set_title(plot_title, fontsize = 'large')
    axs[1].set_title(plot_title, fontsize = 'large')
    axs[2].set_title(plot_title, fontsize = 'large')
    
    # OTHER    
    axs[0].grid()
    axs[1].grid()
    axs[2].grid()
    axs[0].legend()
    axs[1].legend()
    axs[2].legend()
    
    plt.show()
    
    return None